In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/brinkley97/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Read Data

In [2]:
dataset = "../datasets/amazon_reviews_us_Office_Products_v1_00.tsv"
amazon_reviews_copy_df = pd.read_csv(dataset, sep='\t', on_bad_lines='skip', low_memory=False)

## Keep Reviews and Ratings

In [197]:
reviews_ratings_df = amazon_reviews_copy_df.loc[0:, ['star_rating', 'review_body']]
reviews_ratings_df.reset_index(drop=True)

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640249,4,I can't live anymore whithout my Palm III. But...
2640250,4,Although the Palm Pilot is thin and compact it...
2640251,4,This book had a lot of great content without b...
2640252,5,I am teaching a course in Excel and am using t...


In [198]:
reviews_ratings_df['review_body'].dtype

dtype('O')

In [199]:
reviews_ratings_df['review_body'].astype(str)
reviews_ratings_df

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640249,4,I can't live anymore whithout my Palm III. But...
2640250,4,Although the Palm Pilot is thin and compact it...
2640251,4,This book had a lot of great content without b...
2640252,5,I am teaching a course in Excel and am using t...


 ## Select 100000 reviews randomly from positive and negative classes


In [200]:
def update_data_type(df: pd.DataFrame, col_name: str):
    """Update the data type of the star ratings

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with rating values

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the new sentiment appened

    """

    valid_ratings = ['1','2','3','4','5']
    star_rating_series = df[col_name].copy()

    # Convert type to strings
    star_rating_series.astype('str')

    # Check valid list and see which of our stars match
    rows = star_rating_series.index
    is_rating_in_valid_ratings = rows[star_rating_series.isin(valid_ratings)]

    # Convert to list
    is_rating_in_valid_ratings = is_rating_in_valid_ratings.to_list()

    updated_df = df.iloc[is_rating_in_valid_ratings]
    return updated_df

In [201]:
reviews_ratings_df = update_data_type(reviews_ratings_df, 'star_rating')

In [202]:
reviews_ratings_df

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640249,4,I can't live anymore whithout my Palm III. But...
2640250,4,Although the Palm Pilot is thin and compact it...
2640251,4,This book had a lot of great content without b...
2640252,5,I am teaching a course in Excel and am using t...


In [203]:

def separate_reviews_by_rating(df: pd.DataFrame, rating_col: str, threshold: int, sentiment_type: str):
    """Categorizes reviews by adding a rating

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    rating_col: `str`
        Column with rating values
    
    threshold: `int`
        Where to split the ratings such that categories can be formed

    sentiment_type: `str`
        One of three types of sentiment: positive, negative, or neural

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the new sentiment appened
    """


    if sentiment_type == 'positive_sentiment':
        positive_review_threshold = df[rating_col].astype('int32') > threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 1

    elif sentiment_type == 'negative_sentiment':
        positive_review_threshold = df[rating_col].astype('int32') < threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 0

    elif sentiment_type == 'neutral_sentiment':
        positive_review_threshold = df[rating_col].astype('int32') == threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 3
        
    return df

In [204]:
positive_sentiment_df = separate_reviews_by_rating(reviews_ratings_df, 'star_rating', 3, 'positive_sentiment')
positive_sentiment_df

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_8968/4050413545.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sentiment_type] = 1


,star_rating,review_body,positive_sentiment
0,5,Great product.,1
1,5,What's to say about this commodity item except...,1
2,5,"Haven't used yet, but I am sure I will like it.",1
4,4,Gorgeous colors and easy to use,1
5,5,Perfect for planning weekly meals. Removrd the...,1
...,...,...,...
2640249,4,I can't live anymore whithout my Palm III. But...,1
2640250,4,Although the Palm Pilot is thin and compact it...,1
2640251,4,This book had a lot of great content without b...,1
2640252,5,I am teaching a course in Excel and am using t...,1


In [205]:
negative_sentiment_df = separate_reviews_by_rating(reviews_ratings_df, 'star_rating', 3, 'negative_sentiment')
negative_sentiment_df

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_8968/4050413545.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sentiment_type] = 0


,star_rating,review_body,negative_sentiment
3,1,Although this was labeled as &#34;new&#34; the...,0
13,1,worked about a month then died,0
20,1,The phone did not work. No Dial Tone. Not wo...,0
27,1,Not laminated and no reinforced holes for hang...,0
28,1,"Cartridge was over filled, black smears on pap...",0
...,...,...,...
2640139,2,This purchase was intended for a home office s...,0
2640149,2,I bought a Palm V from Amazon and thought it w...,0
2640151,1,The display is excellent - it's a good size an...,0
2640201,1,All the CE based hand held or palm computers h...,0


In [206]:
neutral_sentiment_df = separate_reviews_by_rating(reviews_ratings_df, 'star_rating', 3, 'neutral_sentiment')
neutral_sentiment_df

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_8968/4050413545.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sentiment_type] = 3


,star_rating,review_body,neutral_sentiment
48,3,Nice quality. Happy with the item,3
64,3,The batch I had exploded all over when I tried...,3
95,3,"It is ok, but considering the price plus shipp...",3
133,3,Delighted to receive a sample of these to try ...,3
145,3,I use this light in a dark area of my closet. ...,3
...,...,...,...
2640209,3,I was VERY disappointed to receive my Palm V a...,3
2640219,3,Very basic. The book spends a lot of time des...,3
2640225,3,"Being a Newton devotee, switching to the Palm ...",3
2640234,3,I have a US Robotics Palm Pro (we go back a wa...,3


In [207]:
pos_rand_sampled_df = positive_sentiment_df.sample(100000)
pos_rand_sampled_df

,star_rating,review_body,positive_sentiment
490503,5,great,1
303680,5,Excellent product had them before and glad I c...,1
228023,5,bought these as gift for my workshop participa...,1
82863,5,Love the chalk markers. So easy to use. The ...,1
2188440,5,Another great price!!!!!! WOW! Once again you ...,1
...,...,...,...
919051,5,If you have old slides or negative strips that...,1
2515024,4,This telephone is attractive and works. Some ...,1
2262377,4,It replace the old AT&T phone we had before. W...,1
61867,5,does exactly what I wanted. fast and at a good...,1


In [208]:
neg_rand_sampled_df = negative_sentiment_df.sample(100000)
neg_rand_sampled_df

,star_rating,review_body,negative_sentiment
1714300,1,Even after following the written instructions ...,0
2239861,2,I'm giving this review two stars because the i...,0
589075,2,Just not a fan. Probably is ok for the right p...,0
2560403,1,I used this pencil sharpener in my 5th grade c...,0
322387,2,I accidentally knocked the ruler off the count...,0
...,...,...,...
2501048,2,I just recently purchased this printer and I a...,0
864966,2,"Well, I have 3 boys, and we have been using (n...",0
890945,1,awful!!! the teacher's remote doesn't operate ...,0
2152051,1,I don't print stuff regularly: about once ever...,0


In [209]:
reviews_ratings_df = pd.concat([pos_rand_sampled_df, neg_rand_sampled_df])
reviews_ratings_df

,star_rating,review_body,positive_sentiment,negative_sentiment
490503,5,great,1.0,NaN
303680,5,Excellent product had them before and glad I c...,1.0,NaN
228023,5,bought these as gift for my workshop participa...,1.0,NaN
82863,5,Love the chalk markers. So easy to use. The ...,1.0,NaN
2188440,5,Another great price!!!!!! WOW! Once again you ...,1.0,NaN
...,...,...,...,...
2501048,2,I just recently purchased this printer and I a...,NaN,0.0
864966,2,"Well, I have 3 boys, and we have been using (n...",NaN,0.0
890945,1,awful!!! the teacher's remote doesn't operate ...,NaN,0.0
2152051,1,I don't print stuff regularly: about once ever...,NaN,0.0


In [210]:
pos_sentiment = reviews_ratings_df['positive_sentiment'].dropna()
pos_sentiment

490503     1.0
303680     1.0
228023     1.0
82863      1.0
2188440    1.0
          ... 
919051     1.0
2515024    1.0
2262377    1.0
61867      1.0
2560647    1.0
Name: positive_sentiment, Length: 100000, dtype: float64

In [211]:
neg_sentiment = reviews_ratings_df['negative_sentiment'].dropna()
neg_sentiment

1714300    0.0
2239861    0.0
589075     0.0
2560403    0.0
322387     0.0
          ... 
2501048    0.0
864966     0.0
890945     0.0
2152051    0.0
500735     0.0
Name: negative_sentiment, Length: 100000, dtype: float64

In [212]:
reviews_ratings_df['sentiment'] = pd.concat([pos_sentiment, neg_sentiment])

In [213]:
reviews_ratings_df

,star_rating,review_body,positive_sentiment,negative_sentiment,sentiment
490503,5,great,1.0,NaN,1.0
303680,5,Excellent product had them before and glad I c...,1.0,NaN,1.0
228023,5,bought these as gift for my workshop participa...,1.0,NaN,1.0
82863,5,Love the chalk markers. So easy to use. The ...,1.0,NaN,1.0
2188440,5,Another great price!!!!!! WOW! Once again you ...,1.0,NaN,1.0
...,...,...,...,...,...
2501048,2,I just recently purchased this printer and I a...,NaN,0.0,0.0
864966,2,"Well, I have 3 boys, and we have been using (n...",NaN,0.0,0.0
890945,1,awful!!! the teacher's remote doesn't operate ...,NaN,0.0,0.0
2152051,1,I don't print stuff regularly: about once ever...,NaN,0.0,0.0


In [214]:
reviews_sentiment_df = reviews_ratings_df.drop(columns=['positive_sentiment', 'negative_sentiment'])
reviews_sentiment_df

,star_rating,review_body,sentiment
490503,5,great,1.0
303680,5,Excellent product had them before and glad I c...,1.0
228023,5,bought these as gift for my workshop participa...,1.0
82863,5,Love the chalk markers. So easy to use. The ...,1.0
2188440,5,Another great price!!!!!! WOW! Once again you ...,1.0
...,...,...,...
2501048,2,I just recently purchased this printer and I a...,0.0
864966,2,"Well, I have 3 boys, and we have been using (n...",0.0
890945,1,awful!!! the teacher's remote doesn't operate ...,0.0
2152051,1,I don't print stuff regularly: about once ever...,0.0


In [215]:
reviews_sentiment_df['review_body'].fillna(' ', inplace=True)

# Data Cleaning

## Lower case

In [216]:
def convert_reviews_to_lower_case(df: pd.DataFrame, col_name: str):
    """Convert all reviews to lower case

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the lower cased reviews
    """
    
    lower_case_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    
    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        # print(text_reviews_idx, type(text_review), text_review)

        # NOT all reviews are strings, thus all can't be converted to lower cased
        if type(text_review) != str:
            converted_str = str(text_review)
            # update_text_review = converted_str.lower()
            lower_case_reviews.append(text_review)
            # print(text_reviews_idx, update_text_review)
            # print()
        else:
            update_text_review = text_review.lower()
            lower_case_reviews.append(update_text_review)
            # print(text_reviews_idx, update_text_review)
            # print()

    updated_df['lower_cased'] = lower_case_reviews
    return updated_df

In [217]:
reviews_lower_cased = convert_reviews_to_lower_case(reviews_sentiment_df, 'review_body')

In [218]:
reviews_lower_cased

,star_rating,review_body,sentiment,lower_cased
490503,5,great,1.0,great
303680,5,Excellent product had them before and glad I c...,1.0,excellent product had them before and glad i c...
228023,5,bought these as gift for my workshop participa...,1.0,bought these as gift for my workshop participa...
82863,5,Love the chalk markers. So easy to use. The ...,1.0,love the chalk markers. so easy to use. the ...
2188440,5,Another great price!!!!!! WOW! Once again you ...,1.0,another great price!!!!!! wow! once again you ...
...,...,...,...,...
2501048,2,I just recently purchased this printer and I a...,0.0,i just recently purchased this printer and i a...
864966,2,"Well, I have 3 boys, and we have been using (n...",0.0,"well, i have 3 boys, and we have been using (n..."
890945,1,awful!!! the teacher's remote doesn't operate ...,0.0,awful!!! the teacher's remote doesn't operate ...
2152051,1,I don't print stuff regularly: about once ever...,0.0,i don't print stuff regularly: about once ever...


## Remove HTML and URLs

In [219]:
def remove_html_and_urls(df: pd.DataFrame, col_name: str):
    """Remove HTML and URLs from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the html_and_urls removed
    """
    
    # url_pattern = re.compile(r'https?://\S+|www\. \S+')

    cleaned_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        if isinstance(text_review, str):
            # Check and remove HTML tags
            has_html = bool(re.search('<.*?>', text_review))
            if has_html == True:
                # print("Review", text_reviews_idx, "has HTML -- ", text_review)
                pass

            no_html_review = re.sub('<.*?>', ' ', text_review)
            # print("Review", text_reviews_idx, "without HTML -- ", no_html_review)
        
            # Check and remove URLs
            has_url = bool(re.search(r'http\S+', no_html_review))
            if has_url == True:
                # print("Review", text_reviews_idx, "has URL --", no_html_review)
                pass

            no_html_url_review = re.sub(r'http\S+', '', no_html_review)
            # print("Review", text_reviews_idx, "without HTML, URL -- ", no_html_url_review)
            # print()
            cleaned_reviews.append(no_html_url_review)
        else:
            # print(text_reviews_idx, text_review)
            cleaned_reviews.append(text_review)
            

    updated_df['without_html_urls'] = cleaned_reviews
    return updated_df

In [220]:
no_html_urls_df = remove_html_and_urls(reviews_lower_cased, 'lower_cased')

In [221]:
no_html_urls_df

,star_rating,review_body,sentiment,lower_cased,without_html_urls
490503,5,great,1.0,great,great
303680,5,Excellent product had them before and glad I c...,1.0,excellent product had them before and glad i c...,excellent product had them before and glad i c...
228023,5,bought these as gift for my workshop participa...,1.0,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...
82863,5,Love the chalk markers. So easy to use. The ...,1.0,love the chalk markers. so easy to use. the ...,love the chalk markers. so easy to use. the ...
2188440,5,Another great price!!!!!! WOW! Once again you ...,1.0,another great price!!!!!! wow! once again you ...,another great price!!!!!! wow! once again you ...
...,...,...,...,...,...
2501048,2,I just recently purchased this printer and I a...,0.0,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...
864966,2,"Well, I have 3 boys, and we have been using (n...",0.0,"well, i have 3 boys, and we have been using (n...","well, i have 3 boys, and we have been using (n..."
890945,1,awful!!! the teacher's remote doesn't operate ...,0.0,awful!!! the teacher's remote doesn't operate ...,awful!!! the teacher's remote doesn't operate ...
2152051,1,I don't print stuff regularly: about once ever...,0.0,i don't print stuff regularly: about once ever...,i don't print stuff regularly: about once ever...


## Remove Contractions

In [222]:
store_contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he's": "he is",
    "isn't": "is not",
    "it's": "it is",
    "let's": "let us",
    "mustn't": "must not",
    "shan't": "shall not",
    "she's": "she is",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they're": "they are",
    "wasn't": "was not",
    "we're": "we are",
    "weren't": "were not",
    "won't": "will not",
    "wouldn't": "would not",
    "you're": "you are",
    "you'll": "you will",
    "you'd": "you would",
    "we'll": "we will",
    "we've": "we have",
    "we'd": "we would",
    "I'm": "I am",
    "i've": "I have",
    "I've": "I have",
    "I'd": "I would",
    "it'll": "it will",
    "they'll": "they will",
    "they've": "they have",
    "they'd": "they would",
    "he'll": "he will",
    "he'd": "he would",
    "she'll": "she will",
    "we'd": "we would",
    "we'll": "we will",
    "you've": "you have",
    "you'd": "you would",
    "you'll": "you will",
    "I'll": "I will",
    "I'd": "I would",
    "it's": "it is",
    "it'd": "it would",
    "i'm": "I am",
    "he's": "he is",
    "he'll": "he will",
    "she's": "she is",
    "she'll": "she will",
    "we're": "we are",
    "we've": "we have",
    "we'll": "we will",
    "you're": "you are",
    "you've": "you have",
    "you'll": "you will",
    "they're": "they are",
    "they've": "they have",
    "they'll": "they will",
    "that's": "that is",
    "that'll": "that will",
    "that'd": "that would",
    "who's": "who is",
    "who'll": "who will",
    "who'd": "who would",
    "what's": "what is",
    "what'll": "what will",
    "what'd": "what would",
    "when's": "when is",
    "when'll": "when will",
    "when'd": "when would",
    "where's": "where is",
    "where'll": "where will",
    "where'd": "where would",
    "why's": "why is",
    "why'll": "why will",
    "why'd": "why would",
    "how's": "how is",
    "how'll": "how will",
    "how'd": "how would"
}


In [223]:
def locate_and_replace_contractions(review):
    """Find the contractions to replace from a specific review

    Parameters
    ----------
    review: `str`
        A specific review

    Return
    ------
    non_contraction_review: `str`
        The updated specific review with contractions expanded
    
    """
    if isinstance(review, str):
        get_words = review.split()

        store_non_contraction_words = []

        for word in get_words:
            if word in store_contractions:
                non_contraction_form = store_contractions[word]
                # print(word, "-->", non_contraction_form)

                store_non_contraction_words.append(non_contraction_form)

            else:
                # print(word)
                store_non_contraction_words.append(word)

        non_contraction_review = ' '.join(store_non_contraction_words)
        return non_contraction_review
    else:
        return review


In [224]:
def remove_contractions(df:pd.DataFrame, col_name: str):
    """Remove contractions from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    without_contractions_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        # print("Review", text_reviews_idx, "with possible contraction(s) -- ", text_review)

        without_contraction = locate_and_replace_contractions(text_review)

        # print("Review", text_reviews_idx, "without contraction -- ", without_contraction)
        # print()

        without_contractions_reviews.append(without_contraction)

    updated_df['without_contractions'] = without_contractions_reviews
    return updated_df

In [225]:
no_contractions_df = remove_contractions(no_html_urls_df, 'without_html_urls')

In [226]:
no_contractions_df

,star_rating,review_body,sentiment,lower_cased,without_html_urls,without_contractions
490503,5,great,1.0,great,great,great
303680,5,Excellent product had them before and glad I c...,1.0,excellent product had them before and glad i c...,excellent product had them before and glad i c...,excellent product had them before and glad i c...
228023,5,bought these as gift for my workshop participa...,1.0,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...
82863,5,Love the chalk markers. So easy to use. The ...,1.0,love the chalk markers. so easy to use. the ...,love the chalk markers. so easy to use. the ...,love the chalk markers. so easy to use. the co...
2188440,5,Another great price!!!!!! WOW! Once again you ...,1.0,another great price!!!!!! wow! once again you ...,another great price!!!!!! wow! once again you ...,another great price!!!!!! wow! once again you ...
...,...,...,...,...,...,...
2501048,2,I just recently purchased this printer and I a...,0.0,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...
864966,2,"Well, I have 3 boys, and we have been using (n...",0.0,"well, i have 3 boys, and we have been using (n...","well, i have 3 boys, and we have been using (n...","well, i have 3 boys, and we have been using (n..."
890945,1,awful!!! the teacher's remote doesn't operate ...,0.0,awful!!! the teacher's remote doesn't operate ...,awful!!! the teacher's remote doesn't operate ...,awful!!! the teacher's remote does not operate...
2152051,1,I don't print stuff regularly: about once ever...,0.0,i don't print stuff regularly: about once ever...,i don't print stuff regularly: about once ever...,i do not print stuff regularly: about once eve...


## Remove Non-alphabetical characters

In [227]:
def remove_non_alphabetical_characters(df:pd.DataFrame, col_name: str):
    """Remove Non-alphabetical characters from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the non-alphabetical characters removed
    """

    alphabetical_char_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    # print(text_reviews)

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        
        if isinstance(text_review, str):

            # Check for non-alphabetical characters
            has_non_alphabetical_char = bool(re.search(r'[^a-zA-Z]', text_review))
            if has_non_alphabetical_char == True:
                # print("Review", text_reviews_idx, "has HTML -- ", text_review)
                pass
            
            # Remove non-alphabetical characters
            with_alphabetical_char = re.sub(r'[^a-zA-Z\s]', ' ', text_review)
            # print("Review", text_reviews_idx, "has HTML -- ", with_alphabetical_char)
            alphabetical_char_reviews.append(with_alphabetical_char)
        else:
            alphabetical_char_reviews.append(text_review)

    updated_df['with_alpha_chars_only'] = alphabetical_char_reviews
    return updated_df

In [228]:
only_alpha_chars_df = remove_non_alphabetical_characters(no_contractions_df, 'without_contractions')

In [229]:
only_alpha_chars_df

,star_rating,review_body,sentiment,lower_cased,without_html_urls,without_contractions,with_alpha_chars_only
490503,5,great,1.0,great,great,great,great
303680,5,Excellent product had them before and glad I c...,1.0,excellent product had them before and glad i c...,excellent product had them before and glad i c...,excellent product had them before and glad i c...,excellent product had them before and glad i c...
228023,5,bought these as gift for my workshop participa...,1.0,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...
82863,5,Love the chalk markers. So easy to use. The ...,1.0,love the chalk markers. so easy to use. the ...,love the chalk markers. so easy to use. the ...,love the chalk markers. so easy to use. the co...,love the chalk markers so easy to use the co...
2188440,5,Another great price!!!!!! WOW! Once again you ...,1.0,another great price!!!!!! wow! once again you ...,another great price!!!!!! wow! once again you ...,another great price!!!!!! wow! once again you ...,another great price wow once again you ...
...,...,...,...,...,...,...,...
2501048,2,I just recently purchased this printer and I a...,0.0,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...
864966,2,"Well, I have 3 boys, and we have been using (n...",0.0,"well, i have 3 boys, and we have been using (n...","well, i have 3 boys, and we have been using (n...","well, i have 3 boys, and we have been using (n...",well i have boys and we have been using n...
890945,1,awful!!! the teacher's remote doesn't operate ...,0.0,awful!!! the teacher's remote doesn't operate ...,awful!!! the teacher's remote doesn't operate ...,awful!!! the teacher's remote does not operate...,awful the teacher s remote does not operate...
2152051,1,I don't print stuff regularly: about once ever...,0.0,i don't print stuff regularly: about once ever...,i don't print stuff regularly: about once ever...,i do not print stuff regularly: about once eve...,i do not print stuff regularly about once eve...


## Remove extra spaces

In [230]:
def remove_extra_spaces(df:pd.DataFrame, col_name: str):
    """Remove extra spaces from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    single_spaced_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    # print(text_reviews)

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        if isinstance(text_review, str):
        # Check if there are any extra spaces
            has_extra_space = bool(re.search(r' +', text_review))
            if has_extra_space == True:
                # print("Review", text_reviews_idx, "has extra space -- ", text_review)
                pass
            
            # Remove extra spaces
            single_spaced_review = re.sub(r' +', ' ', text_review)
            # print("Review", text_reviews_idx, "without extra space -- ", single_spaced_review)
            # print()
            
            single_spaced_reviews.append(single_spaced_review)
        else:
            single_spaced_reviews.append(text_review)

    updated_df['without_extra_space'] = single_spaced_reviews
    return updated_df

In [231]:
no_extra_space_df = remove_extra_spaces(only_alpha_chars_df, 'with_alpha_chars_only')

In [232]:
no_extra_space_df

,star_rating,review_body,sentiment,lower_cased,without_html_urls,without_contractions,with_alpha_chars_only,without_extra_space
490503,5,great,1.0,great,great,great,great,great
303680,5,Excellent product had them before and glad I c...,1.0,excellent product had them before and glad i c...,excellent product had them before and glad i c...,excellent product had them before and glad i c...,excellent product had them before and glad i c...,excellent product had them before and glad i c...
228023,5,bought these as gift for my workshop participa...,1.0,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...
82863,5,Love the chalk markers. So easy to use. The ...,1.0,love the chalk markers. so easy to use. the ...,love the chalk markers. so easy to use. the ...,love the chalk markers. so easy to use. the co...,love the chalk markers so easy to use the co...,love the chalk markers so easy to use the colo...
2188440,5,Another great price!!!!!! WOW! Once again you ...,1.0,another great price!!!!!! wow! once again you ...,another great price!!!!!! wow! once again you ...,another great price!!!!!! wow! once again you ...,another great price wow once again you ...,another great price wow once again you have be...
...,...,...,...,...,...,...,...,...
2501048,2,I just recently purchased this printer and I a...,0.0,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...
864966,2,"Well, I have 3 boys, and we have been using (n...",0.0,"well, i have 3 boys, and we have been using (n...","well, i have 3 boys, and we have been using (n...","well, i have 3 boys, and we have been using (n...",well i have boys and we have been using n...,well i have boys and we have been using not ab...
890945,1,awful!!! the teacher's remote doesn't operate ...,0.0,awful!!! the teacher's remote doesn't operate ...,awful!!! the teacher's remote doesn't operate ...,awful!!! the teacher's remote does not operate...,awful the teacher s remote does not operate...,awful the teacher s remote does not operate ac...
2152051,1,I don't print stuff regularly: about once ever...,0.0,i don't print stuff regularly: about once ever...,i don't print stuff regularly: about once ever...,i do not print stuff regularly: about once eve...,i do not print stuff regularly about once eve...,i do not print stuff regularly about once ever...


# Pre-processing

## remove the stop words 

In [233]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [234]:
def filter_stop_words(df:pd.DataFrame, col_name: str):
    """Filter stop words out from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    without_stop_words_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    stop_words = set(stopwords.words("english"))

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        if isinstance(text_review, str):
            text_review_words = word_tokenize(text_review) 

        

            # print("Before stop word removal", text_reviews_idx, " -- ", text_review)

            filtered_review = []

            for text_review_words_idx in range(len(text_review_words)):
                text_review_word = text_review_words[text_review_words_idx]
                
                # Check if review word is a stop word
                if text_review_word in stop_words:
                    # print("  Stop word -- ", text_review_word)
                    pass
                else:
                    # print(text_review_word, " -- is NOT a stop word in review")
                    filtered_review.append(text_review_word)

            
            filtered_review = " ".join(filtered_review)
            # print("After stop word removal", text_reviews_idx, " -- ", filtered_review)
            # print()
            
            without_stop_words_reviews.append(filtered_review)
        else:
            without_stop_words_reviews.append(text_review)
        

    updated_df['without_stop_words'] = without_stop_words_reviews
    return updated_df

In [235]:
no_stop_words_df = filter_stop_words(no_extra_space_df, 'without_extra_space')

In [236]:
no_stop_words_df

,star_rating,review_body,sentiment,lower_cased,without_html_urls,without_contractions,with_alpha_chars_only,without_extra_space,without_stop_words
490503,5,great,1.0,great,great,great,great,great,great
303680,5,Excellent product had them before and glad I c...,1.0,excellent product had them before and glad i c...,excellent product had them before and glad i c...,excellent product had them before and glad i c...,excellent product had them before and glad i c...,excellent product had them before and glad i c...,excellent product glad could find
228023,5,bought these as gift for my workshop participa...,1.0,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...,bought gift workshop participants love love ma...
82863,5,Love the chalk markers. So easy to use. The ...,1.0,love the chalk markers. so easy to use. the ...,love the chalk markers. so easy to use. the ...,love the chalk markers. so easy to use. the co...,love the chalk markers so easy to use the co...,love the chalk markers so easy to use the colo...,love chalk markers easy use colors bright
2188440,5,Another great price!!!!!! WOW! Once again you ...,1.0,another great price!!!!!! wow! once again you ...,another great price!!!!!! wow! once again you ...,another great price!!!!!! wow! once again you ...,another great price wow once again you ...,another great price wow once again you have be...,another great price wow beaten competition tha...
...,...,...,...,...,...,...,...,...,...
2501048,2,I just recently purchased this printer and I a...,0.0,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...,recently purchased printer skeptical called lo...
864966,2,"Well, I have 3 boys, and we have been using (n...",0.0,"well, i have 3 boys, and we have been using (n...","well, i have 3 boys, and we have been using (n...","well, i have 3 boys, and we have been using (n...",well i have boys and we have been using n...,well i have boys and we have been using not ab...,well boys using abusing years maybe every coup...
890945,1,awful!!! the teacher's remote doesn't operate ...,0.0,awful!!! the teacher's remote doesn't operate ...,awful!!! the teacher's remote doesn't operate ...,awful!!! the teacher's remote does not operate...,awful the teacher s remote does not operate...,awful the teacher s remote does not operate ac...,awful teacher remote operate accurately light ...
2152051,1,I don't print stuff regularly: about once ever...,0.0,i don't print stuff regularly: about once ever...,i don't print stuff regularly: about once ever...,i do not print stuff regularly: about once eve...,i do not print stuff regularly about once eve...,i do not print stuff regularly about once ever...,print stuff regularly every months groupon ann...


## perform lemmatization  

- "A sentence with many words"
    - "words" -> word

In [237]:
from nltk.stem import WordNetLemmatizer

def lemmentize_review(df:pd.DataFrame, col_name: str):
    """Lemmentize all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    lemmed_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    lem = WordNetLemmatizer()

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]   
        if isinstance(text_review, str):     
            words_in_review = word_tokenize(text_review) 

            # print("Before lem update", text_reviews_idx, " -- ", text_review)
            # print("Lemmed words", words_in_review)
            

            lemmed_sentence = []

            # Split review into words
            for lemmed_words_idx in range(len(words_in_review)):
                word = words_in_review[lemmed_words_idx]
                
                apply_lemmatization = lem.lemmatize(word)
                # print(apply_lemmatization)
                
                lemmed_sentence.append(apply_lemmatization)
                filtered_review = " ".join(lemmed_sentence)
        
            # print("After lem update -- ", filtered_review)
            # print()

            lemmed_reviews.append(filtered_review)
        else:
            lemmed_reviews.append(text_review)

    updated_df['lemmed_reviews'] = lemmed_reviews

    return updated_df

In [238]:
lemmed_df = lemmentize_review(no_stop_words_df, 'without_stop_words')

In [239]:
lemmed_df

,star_rating,review_body,sentiment,lower_cased,without_html_urls,without_contractions,with_alpha_chars_only,without_extra_space,without_stop_words,lemmed_reviews
490503,5,great,1.0,great,great,great,great,great,great,great
303680,5,Excellent product had them before and glad I c...,1.0,excellent product had them before and glad i c...,excellent product had them before and glad i c...,excellent product had them before and glad i c...,excellent product had them before and glad i c...,excellent product had them before and glad i c...,excellent product glad could find,excellent product glad could find
228023,5,bought these as gift for my workshop participa...,1.0,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...,bought these as gift for my workshop participa...,bought gift workshop participants love love ma...,bought gift workshop participant love love mak...
82863,5,Love the chalk markers. So easy to use. The ...,1.0,love the chalk markers. so easy to use. the ...,love the chalk markers. so easy to use. the ...,love the chalk markers. so easy to use. the co...,love the chalk markers so easy to use the co...,love the chalk markers so easy to use the colo...,love chalk markers easy use colors bright,love chalk marker easy use color bright
2188440,5,Another great price!!!!!! WOW! Once again you ...,1.0,another great price!!!!!! wow! once again you ...,another great price!!!!!! wow! once again you ...,another great price!!!!!! wow! once again you ...,another great price wow once again you ...,another great price wow once again you have be...,another great price wow beaten competition tha...,another great price wow beaten competition tha...
...,...,...,...,...,...,...,...,...,...,...
2501048,2,I just recently purchased this printer and I a...,0.0,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...,i just recently purchased this printer and i a...,recently purchased printer skeptical called lo...,recently purchased printer skeptical called lo...
864966,2,"Well, I have 3 boys, and we have been using (n...",0.0,"well, i have 3 boys, and we have been using (n...","well, i have 3 boys, and we have been using (n...","well, i have 3 boys, and we have been using (n...",well i have boys and we have been using n...,well i have boys and we have been using not ab...,well boys using abusing years maybe every coup...,well boy using abusing year maybe every couple...
890945,1,awful!!! the teacher's remote doesn't operate ...,0.0,awful!!! the teacher's remote doesn't operate ...,awful!!! the teacher's remote doesn't operate ...,awful!!! the teacher's remote does not operate...,awful the teacher s remote does not operate...,awful the teacher s remote does not operate ac...,awful teacher remote operate accurately light ...,awful teacher remote operate accurately light ...
2152051,1,I don't print stuff regularly: about once ever...,0.0,i don't print stuff regularly: about once ever...,i don't print stuff regularly: about once ever...,i do not print stuff regularly: about once eve...,i do not print stuff regularly about once eve...,i do not print stuff regularly about once ever...,print stuff regularly every months groupon ann...,print stuff regularly every month groupon annu...


# TF-IDF Feature Extraction

In [240]:
def tf_idf_feature_extraction(df: pd.DataFrame, col_name: str):
    """Extract the TF-IDF features from the reviews.

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    tf_idf_features:
        A matrix containing the TF-IDF features extracted
        
    """
    
    vectorizer = TfidfVectorizer()
    tf_idf_features = vectorizer.fit_transform(df[col_name])

    return tf_idf_features


In [242]:
tf_idf_features = tf_idf_feature_extraction(lemmed_df, 'lemmed_reviews')


In [243]:
tf_idf_features[0]

<1x56819 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

## Split Features and Sentiment Labels

In [244]:
sentiments = lemmed_df['sentiment']
sentiments.shape

(200000,)

In [245]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tf_idf_features, sentiments, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((160000, 56819), (40000, 56819), (160000,), (40000,))

# Models + Evaluation Metrics

In [246]:
import sklearn
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

In [247]:
def eval_accuracy(y_true, y_prediction):
    return sklearn.metrics.accuracy_score(y_true, y_prediction)

def eval_precision(y_true, y_prediction):
    return sklearn.metrics.precision_score(y_true, y_prediction)

def eval_recall(y_true, y_prediction):
    return sklearn.metrics.recall_score(y_true, y_prediction)

def eval_f1_score(y_true, y_prediction):
    return sklearn.metrics.f1_score(y_true, y_prediction)

In [248]:
def train_eval_metric(y_train_true, y_train_predictions):
    accuracy = eval_accuracy(y_train_true, y_train_predictions)
    precision = eval_precision(y_train_true, y_train_predictions)
    recall = eval_recall(y_train_true, y_train_predictions)
    f1 = eval_f1_score(y_train_true, y_train_predictions)

    metrics_dict = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

    return metrics_dict

def test_eval_metric(y_test_true, y_test_predictions):
    accuracy = eval_accuracy(y_test_true, y_test_predictions)
    precision = eval_precision(y_test_true, y_test_predictions)
    recall = eval_recall(y_test_true, y_test_predictions)
    f1 = eval_f1_score(y_test_true, y_test_predictions)

    metrics_dict = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

    return metrics_dict

# Perceptron

In [249]:
def perceptron_model(X_train, X_test, y_train, y_test): 

    technique = Perceptron(tol=1e-3, random_state=0)
    technique.fit(X_train, y_train)
    y_train_predictions = technique.predict(X_train)
    y_test_predictions = technique.predict(X_test)


    train_metrics = train_eval_metric(y_train, y_train_predictions)
    test_metrics = test_eval_metric(y_test, y_test_predictions)

    return train_metrics, test_metrics


In [250]:
perceptron_train_metrics, perceptron_test_metrics = perceptron_model(X_train, X_test, y_train, y_test)

In [251]:
perceptron_train_metrics, perceptron_test_metrics

({'Accuracy': 0.8931875,
  'Precision': 0.9150690715256429,
  'Recall': 0.8668491506993138,
  'F1 Score': 0.8903066791614783},
 {'Accuracy': 0.844825,
  'Precision': 0.8665709423526909,
  'Recall': 0.8150352623418197,
  'F1 Score': 0.8400134030981777})

# SVM

In [252]:
def svm_model(X_train, X_test, y_train, y_test): 

    technique = LinearSVC(tol=1e-3, random_state=0)
    technique.fit(X_train, y_train)
    y_train_predictions = technique.predict(X_train)
    y_test_predictions = technique.predict(X_test)


    train_metrics = train_eval_metric(y_train, y_train_predictions)
    test_metrics = test_eval_metric(y_test, y_test_predictions)

    return train_metrics, test_metrics


In [253]:
svm_train_metrics, svm_test_metrics = svm_model(X_train, X_test, y_train, y_test)

/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [254]:
svm_train_metrics, svm_test_metrics

({'Accuracy': 0.93013125,
  'Precision': 0.9315451558698868,
  'Recall': 0.928506255702626,
  'F1 Score': 0.9300232233510481},
 {'Accuracy': 0.8907,
  'Precision': 0.8929811320754717,
  'Recall': 0.8877106987445605,
  'F1 Score': 0.8903381157820809})

# Logistic Regression

In [255]:
def logistic_regression_model(X_train, X_test, y_train, y_test): 

    technique = LogisticRegression(random_state=0)
    technique.fit(X_train, y_train)
    y_train_predictions = technique.predict(X_train)
    y_test_predictions = technique.predict(X_test)


    train_metrics = train_eval_metric(y_train, y_train_predictions)
    test_metrics = test_eval_metric(y_test, y_test_predictions)

    return train_metrics, test_metrics

In [256]:
logistic_regression_train_metrics, logistic_regression_test_metrics = logistic_regression_model(X_train, X_test, y_train, y_test)

In [257]:
logistic_regression_train_metrics, logistic_regression_test_metrics

({'Accuracy': 0.9094125,
  'Precision': 0.9123166970860344,
  'Recall': 0.9059082330296099,
  'F1 Score': 0.9091011715124301},
 {'Accuracy': 0.895175,
  'Precision': 0.8983059392961581,
  'Recall': 0.8911619066673335,
  'F1 Score': 0.8947196625404876})

# Naive Bayes

In [258]:
def naive_bayes_model(X_train, X_test, y_train, y_test): 

    technique = MultinomialNB()
    technique.fit(X_train.toarray(), y_train)
    y_train_predictions = technique.predict(X_train)
    y_test_predictions = technique.predict(X_test)

    train_metrics = train_eval_metric(y_train, y_train_predictions)
    test_metrics = test_eval_metric(y_test, y_test_predictions)

    return train_metrics, test_metrics


In [259]:
naive_bayes_train_metrics, naive_bayes_test_metrics = naive_bayes_model(X_train, X_test, y_train, y_test)

In [260]:
naive_bayes_train_metrics, naive_bayes_test_metrics

({'Accuracy': 0.8752,
  'Precision': 0.8840038375439718,
  'Recall': 0.8637619208319272,
  'F1 Score': 0.8737656623383192},
 {'Accuracy': 0.8577,
  'Precision': 0.8665983081261215,
  'Recall': 0.8454459060671234,
  'F1 Score': 0.8558914375411413})